In [6]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from astropy.io import fits
import matplotlib.pyplot as plt
from scipy.signal import medfilt
from scipy.interpolate import interp1d

In [7]:
# Decontinuous spectrum parameters
param = {'poly_global_order':5,
         'nor':1,
         'poly_lowerlimit':3,
         'poly_upperlimit':4,
         'median_radius':3, 
         'poly_SM':0,
         'poly_del_filled':2
        } 

def csp_polyfit(sp,angs,param):
           
    # standardize flux
    sp_c = np.mean(sp)                 
    sp = sp - sp_c                     
    sp_s = np.std(sp)  
    sp = sp / sp_s
    
    # standardize wavelength
    angs_c = np.mean(angs)
    angs = angs - angs_c
    angs_s = np.std(angs)
    angs = angs/angs_s
    
    param['poly_sp_c'] = sp_c
    param['poly_sp_s'] = sp_s
    param['poly_angs_c'] = angs_c
    param['poly_angs_s'] = angs_s
    
    data_flag = np.full(sp.shape, 1)
    
    i = 0
    con = True
    while(con):
        P_g = np.polyfit(angs, sp, param['poly_global_order'])  # coefficient
        param['poly_P_g'] = P_g
        fitval_1 = np.polyval(P_g, angs) 
        dev = fitval_1 - sp
        sig_g = np.std(dev)
        
        data_flag_new = (dev > (-param['poly_upperlimit'] * sig_g)) * (dev < (param['poly_lowerlimit'] * sig_g))
    
        if sum(abs( data_flag_new - data_flag ))>0:
            if param['poly_del_filled'] == 1: 
                data_flag = data_flag_new
            else:
                fill_flag = data_flag - data_flag_new
                index_1 = np.where(fill_flag != 0)
                sp[index_1] = fitval_1[index_1]
        else:
            con = False
        i += 1
    
    index_2 = np.where(data_flag != 0)
    param['poly_sp_filtered'] = sp[index_2]
    param['poly_angs_filtered'] = angs[index_2]
    
    return param

def sp_median_polyfit1stage(flux,lambda_log,param):
    flux1 = flux
    lambda1 = lambda_log

    # Median filter
    flux_median1 = medfilt(flux1, param['median_radius']) 

    dev1 = flux_median1 - flux1
    sigma = np.std(dev1)
    data_flag1 = (dev1 < (param['poly_lowerlimit'] * sigma)) * (dev1 > (-param['poly_upperlimit'] * sigma))
    
    fill_flag1 = 1 - data_flag1
    
    if param['poly_del_filled'] == 1:
        index_1 = np.where(data_flag1)
        flux1 = flux1[index_1]
        lambda1 = lambda1[index_1]
    elif param['poly_del_filled'] == 2:
        index_2 = np.where(fill_flag1)
        flux1[index_2] = flux_median1[index_2]

        param = csp_polyfit(flux1, lambda1, param)

        
    angs = lambda1 - param['poly_angs_c']
    angs = angs / param['poly_angs_s']

    fitval_g = np.polyval(param['poly_P_g'], angs)
    continum_fitted = fitval_g * param['poly_sp_s'] + param['poly_sp_c']
    if param['poly_SM'] ==1: 
        angss = lambda1
    else: 
        angss = 10 ** lambda1
    return continum_fitted
